<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.05/IR_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install pandas nltk

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# download the files from github

pro_israel_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/pro-israel.txt?raw=true"
pro_palestine_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/pro-palestine.txt?raw=true"
anti_israel_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/anti-israel.txt?raw=true"
anti_palestine_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/Word/anti-palestine.txt?raw=true"

sentences_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/sentences/sentences.csv?raw=true"

In [6]:
# download all the file into the same dir as the note book

import urllib.request

# Download files
urllib.request.urlretrieve(pro_israel_url, 'pro-israel.txt')
urllib.request.urlretrieve(pro_palestine_url, 'pro-palestine.txt')
urllib.request.urlretrieve(anti_israel_url, 'anti-israel.txt')
urllib.request.urlretrieve(anti_palestine_url, 'anti-palestine.txt')
urllib.request.urlretrieve(sentences_url, 'sentences.csv')

print("Files downloaded successfully!")

Files downloaded successfully!


In [15]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

# Download the punkt tokenizer data
nltk.download('punkt')
nltk.download('punkt_tab')

stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [8]:
# Function to load and stem words from a file
def load_and_stem_words(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    stemmed_words = {stemmer.stem(word) for word in words}
    return stemmed_words

In [9]:
# Load and stem words for each class
pro_israel_words = load_and_stem_words('pro-israel.txt')
anti_palestine_words = load_and_stem_words('anti-palestine.txt')
pro_palestine_words = load_and_stem_words('pro-palestine.txt')
anti_israel_words = load_and_stem_words('anti-israel.txt')

# Load sentences from the CSV file
sentences_df = pd.read_csv('sentences.csv')

In [10]:
def classify_sentence(sentence):
    # Tokenize and stem the sentence
    tokens = word_tokenize(sentence.lower())
    stemmed_tokens = {stemmer.stem(token) for token in tokens}

    # Initialize the one-hot encoded vector
    # [pro-israel, pro-palestine, neutral, anti-israel, anti-palestine]
    vector = [0, 0, 0, 0, 0]

    # Check for each class
    if stemmed_tokens & pro_israel_words:
        vector[0] = 1  # pro-israel
    if stemmed_tokens & pro_palestine_words:
        vector[1] = 1  # pro-palestine
    if stemmed_tokens & anti_israel_words:
        vector[3] = 1  # anti-israel
    if stemmed_tokens & anti_palestine_words:
        vector[4] = 1  # anti-palestine

    # Check if the sentence is neutral (no words from any class)
    if not any(vector):
        vector[2] = 1  # neutral

    return vector

In [20]:
def one_hot_to_string(one_hot_vector):
    """
    Converts a one-hot encoded vector to a string representation of the classes.

    Args:
        one_hot_vector (list): A list of 5 integers representing the one-hot encoded vector.
                              Format: [pro-israel, pro-palestine, neutral, anti-israel, anti-palestine]

    Returns:
        str: A string representation of the classes, e.g., "pro-israel, anti-palestine".
    """
    classes = ["pro-israel", "pro-palestine", "neutral", "anti-israel", "anti-palestine"]
    active_classes = [classes[i] for i, value in enumerate(one_hot_vector) if value == 1]
    return ", ".join(active_classes) if active_classes else "neutral"


def string_to_one_hot(class_string):
    """
    Converts a string representation of the classes to a one-hot encoded vector.

    Args:
        class_string (str): A string representation of the classes, e.g., "pro-israel, anti-palestine".

    Returns:
        list: A one-hot encoded vector, e.g., [1, 0, 0, 0, 1].
    """
    classes = ["pro-israel", "pro-palestine", "neutral", "anti-israel", "anti-palestine"]
    one_hot_vector = [0, 0, 0, 0, 0]

    # Split the input string by commas and strip whitespace
    active_classes = [c.strip() for c in class_string.split(",")]

    # Set the corresponding indices to 1
    for c in active_classes:
        if c in classes:
            one_hot_vector[classes.index(c)] = 1

    # Handle neutral case (if no classes are active)
    if not any(one_hot_vector):
        one_hot_vector[2] = 1  # neutral

    return one_hot_vector

'N\\A'

In [18]:
sentences_df.head()

,id,sentence
0,aj_1,pope renews call for gaza ceasefire release o...
1,aj_1,Pope Francis says his thoughts go to those fac...
2,aj_1,pope francis has renewed calls for an immediat...
3,aj_2,biden is still the best us president israel co...
4,aj_2,united states president ronald reagans order t...


In [16]:
# Apply classification to each sentence
sentences_df['classification'] = sentences_df['sentence'].apply(classify_sentence)

AttributeError: 'float' object has no attribute 'lower'